# The Purge

## Libraries and Settings

In [11]:
# Libraries
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Importing Data

In [13]:
# Read the data to a pandas data frame
df_rot = pd.read_csv('raw_data/coop_rot.csv', sep=',', encoding='utf-8')
df_weiss = pd.read_csv('raw_data/coop_weiss.csv', sep=',', encoding='utf-8')
df_champagner = pd.read_csv('raw_data/coop_champagner.csv', sep=',', encoding='utf-8')
df_dessert = pd.read_csv('raw_data/coop_dessert.csv', sep=',', encoding='utf-8')
df_rose = pd.read_csv('raw_data/coop_rose.csv', sep=',', encoding='utf-8')
df_schaumweine = pd.read_csv('raw_data/coop_schaumweine.csv', sep=',', encoding='utf-8')